## Preprocessing

In [ ]:
import numpy as np 
import os
import pandas as pd

In [ ]:
data_dir = "gene_expression/gene_expression_files"  
mapping_file = "gdc_mapping.tsv"  

#filter for .tsv-- these will be the gene expression files only
mapping = pd.read_csv(mapping_file, sep="\t")
mapping = mapping[mapping["Sample Type"] == "Primary Tumor"] #important-- this step was missed. first i just filtered for "tumor" and it gave other things in the cohort.
tsv_files = mapping[mapping["File Name"].str.endswith(".tsv")]
expression_data = {}

for _, row in tsv_files.iterrows():
    file_name = row["File Name"]  # Exact filename with extension-- the extension is included in the file name
    sample_id = row["Sample ID"]
    file_path = os.path.join(data_dir, file_name)

    if os.path.exists(file_path):
        try:
            # Read the first 5 rows to confirm column format, skipping the first row (header=1). Was getting weird skipping at first. 
            exp_levels = pd.read_csv(file_path, sep="\t", header=1, nrows=5)
            required_cols = {"gene_name", "gene_type", "tpm_unstranded"}
            if not required_cols.issubset(exp_levels.columns):
                print(f"Skipping {file_name}: Missing expected columns.")
                continue
            exp_levels = pd.read_csv(file_path, sep="\t", header=1, usecols=required_cols)
            exp_levels = exp_levels[exp_levels["gene_type"] == "protein_coding"]
            exp_levels = exp_levels[["gene_name", "tpm_unstranded"]].set_index("gene_name")
            expression_data[sample_id] = exp_levels["tpm_unstranded"]
        
        except Exception as e:
            print(f"Error reading {file_name}: {e}")
    else:
        print(f"Warning: File {file_name} not found in {data_dir}")

expression_matrix = pd.DataFrame(expression_data)
expression_matrix = expression_matrix.sort_index().astype(float)

output_file = "gene_expression_matrix.tsv"
expression_matrix.to_csv(output_file, sep="\t")

print(f"Gene expression matrix saved as {output_file}")    

In [ ]:
mapping_dict = dict(zip(mapping_df['File Name'], mapping_df['Sample ID']))

def extract_mutations(filepath):
    try:
        df = pd.read_csv(filepath, low_memory=False)
        non_silent = df[~df['Variant_Classification'].isin([
            'Silent', 'Intron', 'IGR', '3\'UTR', '5\'UTR', '3\'Flank', '5\'Flank', 'RNA', 'lincRNA'])]
        return non_silent[['Tumor_Sample_Barcode', 'Hugo_Symbol']].dropna()
    except:
        return pd.DataFrame(columns=['Tumor_Sample_Barcode', 'Hugo_Symbol'])

all_mutations = []

for filepath in glob(os.path.join("extracted/somatic_mutations", "*.maf.csv")):
    # Get the file name base (no directory, no extension)
    base_name = os.path.basename(filepath).replace(".maf.csv", "")
    mapped_filename = f"{base_name}.maf.gz"

    sample_id = mapping_dict.get(mapped_filename)
    if not sample_id:
        print(f"Warning: {mapped_filename} not found in mapping file.")
        continue

    df = extract_mutations(filepath)
    if not df.empty:
        df['Sample_ID'] = sample_id
        all_mutations.append(df[['Sample_ID', 'Hugo_Symbol']])
combined = pd.concat(all_mutations).drop_duplicates()
combined['value'] = 1
binary_matrix = combined.pivot_table(index='Hugo_Symbol', columns='Sample_ID', values='value', fill_value=0)
binary_matrix.to_csv("somatic_mutation_matrix_by_sample_id.csv")


In [ ]:
print("gene expression")
expression_df = pd.read_csv("gene_expression_matrix.tsv", index_col=0, sep="\t")
print(f"Expression matrix shape: {expression_df.shape}")
print(f"Expression matrix columns (samples): {expression_df.columns[:5]}")  
print(f"Expression matrix index (genes) sample: {expression_df.index[:5]}")  

print("\nsomatic mutation matrix")
mutation_df = pd.read_csv("somatic_mutation_matrix_by_sample_id.csv", index_col=0)
print(f"Mutation matrix shape: {mutation_df.shape}")
print(f"Mutation matrix columns (samples): {mutation_df.columns[:5]}")
print(f"Mutation matrix index (genes) sample: {mutation_df.index[:5]}")

mutation_df.columns = mutation_df.columns.str.strip()

expression_df = expression_df[~expression_df.index.duplicated(keep='first')]
mutation_df = mutation_df[~mutation_df.index.duplicated(keep='first')]
print("\nDropped duplicate genes if any.")

expression_df.index.name = "gene_name"
mutation_df.index.name = "gene_name"

print("\nBefore stripping last 3 chars from sample IDs:")
print(f"Expression columns example: {list(expression_df.columns[:5])}")
print(f"Mutation columns example: {list(mutation_df.columns[:5])}")

expression_df.columns = [col[:-3] if len(col) > 3 else col for col in expression_df.columns]
mutation_df.columns = [col[:-3] if len(col) > 3 else col for col in mutation_df.columns]
dup_expr = expression_df.columns.duplicated()
dup_mut = mutation_df.columns.duplicated()

print(f"\nDuplicate sample IDs in expression: {expression_df.columns[dup_expr].tolist()}")
print(f"Duplicate sample IDs in mutation: {mutation_df.columns[dup_mut].tolist()}")

expression_df = expression_df.loc[:, ~dup_expr]
mutation_df = mutation_df.loc[:, ~dup_mut]

print("\nAfter stripping last 3 chars from sample IDs (Case IDs):")
print(f"Expression columns example: {list(expression_df.columns[:5])}")
print(f"Mutation columns example: {list(mutation_df.columns[:5])}")

all_genes = expression_df.index

mutation_aligned = mutation_df.reindex(all_genes, fill_value=0)

common_samples = expression_df.columns.intersection(mutation_aligned.columns)
expression_filtered = expression_df[common_samples]
mutation_filtered = mutation_aligned[common_samples]

print(f"Number of genes (expression): {expression_filtered.shape[0]}")
print(f"Number of samples: {len(common_samples)}")
expression_T = expression_filtered.T
mutation_T = mutation_filtered.T

print(f"Transposed expression shape: {expression_T.shape}")
print(f"Transposed mutation shape: {mutation_T.shape}")

merged_df = pd.concat([expression_T, mutation_T], axis=1)
print(f"Merged matrix shape (samples × features): {merged_df.shape}")
merged_df.to_csv("merged_expression_mutation.csv")
print("\nMerged matrix saved as 'merged_expression_mutation.csv'")


## Autoencoder

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight
import tensorflow.keras.backend as K

# Log-transform and scale from 0-1
expression_data_log = np.log1p(expression_T)
scaler_expr = MinMaxScaler()
expression_scaled = scaler_expr.fit_transform(expression_data_log)
# Mutation data as-is (0/1)
y_flat = mutation_T.values.flatten()
n_positives = np.sum(y_flat == 1)
n_negatives = np.sum(y_flat == 0)
#pos_weight = n_negatives / (n_positives + 1e-6)  # Avoid divide by zero
#print(pos_weight)
#pos_weight = min(pos_weight, 10)
#print(pos_weight)
mutation_binary = mutation_T.astype(np.float32)
# Flatten binary mutation matrix

def build_autoencoder(input_dim, encoding_dim=200):
    input_layer = keras.Input(shape=(input_dim,))
    encoded = layers.Dense(800, activation='relu')(input_layer)
    encoded = layers.Dense(400, activation='relu')(encoded)
    encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
    
    
    decoded = layers.Dense(400, activation='relu')(encoded)
    decoded = layers.Dense(800, activation='relu')(decoded)
    decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)
    autoencoder = keras.Model(input_layer, decoded)
    encoder = keras.Model(input_layer, encoded)
    autoencoder.compile(optimizer=Adam(learning_rate=1e-4), loss='mse')
    return autoencoder, encoder

#def weighted_binary_crossentropy(pos_weight):
    #def loss(y_true, y_pred):
        #epsilon = K.epsilon()
        #y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        #logloss = -(pos_weight * y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred))
        #return K.mean(logloss, axis=-1)
    #return loss


def build_mutation_autoencoder(input_dim, encoding_dim=200, dropout_rate=0.4):
    input_layer = keras.Input(shape=(input_dim,))
    x = layers.Dense(800, activation='relu')(input_layer)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(400, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    encoded = layers.Dense(encoding_dim, activation='relu')(x)
    
    x = layers.Dense(400, activation='relu')(encoded)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(800, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    decoded = layers.Dense(input_dim, activation='sigmoid')(x)

    autoencoder = keras.Model(inputs=input_layer, outputs=decoded)
    encoder = keras.Model(inputs=input_layer, outputs=encoded)
    autoencoder.compile(optimizer=Adam(learning_rate=5e-5, decay=1e-5), loss="binary_crossentropy")
    return autoencoder, encoder


#Gene expression
autoencoder_expr, encoder_expr = build_autoencoder(expression_scaled.shape[1], encoding_dim=200)
autoencoder_expr.fit(expression_scaled, expression_scaled,
                     epochs=50,
                     batch_size=16,
                     validation_split=0.25,
                     shuffle=True)

#Somatic mutation 
autoencoder_mut, encoder_mut = build_mutation_autoencoder(mutation_binary.shape[1], encoding_dim=200)
autoencoder_mut.fit(mutation_binary, mutation_binary,
                    epochs=50,
                    batch_size=8,
                    validation_split=0.25,
                    shuffle=True,)

latent_expr = encoder_expr.predict(expression_scaled)
latent_mut = encoder_mut.predict(mutation_binary)
latent_combined = np.hstack([latent_expr, latent_mut])

print(f"Combined latent features shape: {latent_combined.shape}")


In [ ]:
encoded_df = pd.DataFrame(latent_combined, index=merged_df.index, columns=[f"latent_{i}" for i in range(latent_combined.shape[1])])
encoded_df.to_csv("encoder_output__400_final.csv")

In [ ]:
from scipy.stats import pearsonr
expr_recon = autoencoder_expr.predict(expression_scaled)
pearsons = []

for i in range(expression_scaled.shape[0]):
    r, _ = pearsonr(expression_scaled[i], expr_recon[i])
    pearsons.append(r)

print(f"Mean Pearson: {np.mean(pearsons):.2f}")
print(f"Median Pearson: {np.median(pearsons):.2f}")


## Clustering

In [ ]:
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
MIN_CLUSTER_SIZE = 20
BIC_RANGE = range(2, 10)
RANDOM_STATE = 42
REG_COVAR = 1e-3

def bic_optimal_n(X, n_range=BIC_RANGE):
    bics = []
    for n in n_range:
        if len(X) <= n:
            break
        gmm = GaussianMixture(
            n_components=n,
            random_state=RANDOM_STATE,
            reg_covar=REG_COVAR
        )
        gmm.fit(X)
        bics.append(gmm.bic(X))
    if not bics:
        return 1
    return n_range[np.argmin(bics)]

def recursive_gmm(encoded_df, min_size=MIN_CLUSTER_SIZE):
    df = encoded_df.copy()

    # Level 1
    opt1 = bic_optimal_n(df)
    print(f"Optimal clusters (Level 1): {opt1}")

    gmm1 = GaussianMixture(
        n_components=opt1,
        random_state=RANDOM_STATE,
        reg_covar=REG_COVAR
    ).fit(df)
    df["cluster_1"] = gmm1.predict(df)

    # Level 2
    cluster_counter = 0
    df["cluster_2"] = np.nan

    for c1 in sorted(df["cluster_1"].unique()):
        subset_idx = df.index[df["cluster_1"] == c1]
        subset = df.loc[subset_idx, df.columns.difference(["cluster_1","cluster_2"], sort=False)]

        if len(subset) < min_size * 2:
            df.loc[subset_idx, "cluster_2"] = cluster_counter
            cluster_counter += 1
            continue

        opt2 = bic_optimal_n(subset)
        gmm2 = GaussianMixture(
            n_components=opt2,
            random_state=RANDOM_STATE,
            reg_covar=REG_COVAR
        ).fit(subset)
        sub_labels = gmm2.predict(subset)

        for sub in np.unique(sub_labels):
            child_idx = subset_idx[sub_labels == sub]
            if len(child_idx) < min_size:
                #merging. 
                df.loc[child_idx, "cluster_2"] = cluster_counter
            else:
                df.loc[child_idx, "cluster_2"] = cluster_counter
            cluster_counter += 1

    # Fixing lables
    df["cluster_2"] = df["cluster_2"].astype("Int64")
    valid = df["cluster_2"].dropna().unique()
    label_map = {old: i+1 for i, old in enumerate(sorted(valid))}
    df["cluster_final"] = df["cluster_2"].map(label_map)
    df["cluster_label"] = df["cluster_final"].apply(lambda x: f"C{x}" if pd.notna(x) else "unassigned")

    print("\nCluster distribution:")
    print(df["cluster_label"].value_counts())

    return df


In [ ]:
def extend_clustering(df, feature_cols, start_label_int):
    """Split each existing cluster (C1–Cn) into subclusters, returning new df and next starting int."""
    current_labels = sorted(df["cluster_label"].unique())
    new_label_counter = start_label_int
    df["cluster_next"] = np.nan

    for label in current_labels:
        sub_idx = df.index[df["cluster_label"] == label]
        subset = df.loc[sub_idx, feature_cols]

        if len(subset) < MIN_CLUSTER_SIZE * 2:
            df.loc[sub_idx, "cluster_next"] = new_label_counter
            new_label_counter += 1
            continue

        opt = bic_optimal_n(subset)
        gmm = GaussianMixture(
            n_components=opt,
            random_state=RANDOM_STATE,
            reg_covar=REG_COVAR
        ).fit(subset)
        sub_labels = gmm.predict(subset)

        for sub in np.unique(sub_labels):
            child_idx = sub_idx[sub_labels == sub]
            if len(child_idx) < MIN_CLUSTER_SIZE:
                df.loc[child_idx, "cluster_next"] = new_label_counter
            else:
                df.loc[child_idx, "cluster_next"] = new_label_counter
            new_label_counter += 1

    # labels
    df["cluster_next"] = df["cluster_next"].astype("Int64")
    valid = df["cluster_next"].dropna().unique()
    label_map = {old: i+1 for i, old in enumerate(sorted(valid))}
    df["cluster_final"] = df["cluster_next"].map(label_map)
    df["cluster_label"] = df["cluster_final"].apply(lambda x: f"C{x}" if pd.notna(x) else "unassigned")

    print("\nCluster distribution after next level:")
    print(df["cluster_label"].value_counts())

    return df, new_label_counter


In [ ]:
feature_cols = encoded_df.columns  
clustered_df, next_label = extend_clustering(clustered_df, feature_cols, start_label_int=5)


In [76]:
def refine_clusters(df, feature_cols, target_clusters, start_label_int):
    """Split only selected clusters; keep all other labels unchanged."""
    from sklearn.mixture import GaussianMixture
    import numpy as np
    import pandas as pd

    next_label = start_label_int
    df["cluster_next"] = np.nan

    for label in df["cluster_label"].unique():
        sub_idx = df.index[df["cluster_label"] == label]
        subset = df.loc[sub_idx, feature_cols]

        # skip clusters not in target list
        if label not in target_clusters:
            df.loc[sub_idx, "cluster_next"] = next_label
            next_label += 1
            continue

        if len(subset) < MIN_CLUSTER_SIZE * 2:
            df.loc[sub_idx, "cluster_next"] = next_label
            next_label += 1
            continue

        opt = bic_optimal_n(subset)
        gmm = GaussianMixture(
            n_components=opt,
            random_state=RANDOM_STATE,
            reg_covar=REG_COVAR
        ).fit(subset)
        sub_labels = gmm.predict(subset)

        for sub in np.unique(sub_labels):
            child_idx = sub_idx[sub_labels == sub]
            if len(child_idx) < MIN_CLUSTER_SIZE:
                df.loc[child_idx, "cluster_next"] = next_label
            else:
                df.loc[child_idx, "cluster_next"] = next_label
            next_label += 1

    # clean labels
    df["cluster_next"] = df["cluster_next"].astype("Int64")
    valid = df["cluster_next"].dropna().unique()
    label_map = {old: i + 1 for i, old in enumerate(sorted(valid))}
    df["cluster_final"] = df["cluster_next"].map(label_map)
    df["cluster_label"] = df["cluster_final"].apply(lambda x: f"C{x}" if pd.notna(x) else "unassigned")

    print("\nCluster distribution after refinement:")
    print(df["cluster_label"].value_counts().sort_index())

    return df, next_label


In [ ]:
feature_cols = encoded_df.columns 
target_clusters = ["C4", "C7"]   
clustered_df, next_label = refine_clusters(
    clustered_df,
    feature_cols,
    target_clusters,
    start_label_int=8  #currently 7 clusters
)


In [ ]:
clustered_df["cluster_label"].value_counts().sum()


In [ ]:
import umap
import matplotlib.pyplot as plt
import seaborn as sns

reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(encoded_df)
umap_df = pd.DataFrame(embedding, columns=["UMAP1", "UMAP2"])
umap_df["cluster"] = clustered_df["cluster_label"].values

plt.figure(figsize=(6,5))
sns.scatterplot(data=umap_df, x="UMAP1", y="UMAP2", hue="cluster", s=20, palette="tab10")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.title("UMAP colored by clusters")
plt.show()


In [ ]:
feature_cols = encoded_df.columns
target_clusters = ["C3", "C7"]   
clustered_df, next_label = refine_clusters(
    clustered_df,
    feature_cols,
    target_clusters,
    start_label_int=10    
)


In [ ]:
from sklearn.metrics import pairwise_distances

def merge_small_clusters(df, feature_cols, min_size=20):
    """
    I tried merging clusters smaller than min_size into the nearest larger cluster
    using distance between centroids in space. 
    This ended up being pointless becaues they went
    to their parent cluster anyways...
    """
    df = df.copy()
    cluster_sizes = df["cluster_label"].value_counts()
    small_clusters = cluster_sizes[cluster_sizes < min_size].index.tolist()
    large_clusters = cluster_sizes[cluster_sizes >= min_size].index.tolist()
    if not small_clusters:
        print("No small clusters to merge.")
        return df

    print(f"Merging clusters smaller than {min_size}: {small_clusters}")
    
    centroids = (
        df.groupby("cluster_label")[feature_cols]
        .mean()
        .loc[cluster_sizes.index]
    )

    for small in small_clusters:
        dists = pairwise_distances(
            centroids.loc[[small]], centroids.loc[large_clusters]
        )[0]
        nearest_large = large_clusters[np.argmin(dists)]
        print(f"→ {small} merged into {nearest_large}")
        
        #Assigning Samples
        df.loc[df["cluster_label"] == small, "cluster_label"] = nearest_large
    df["cluster_label"] = (
        df["cluster_label"]
        .astype("category")
        .cat.rename_categories(
            {old: f"C{i+1}" for i, old in enumerate(sorted(df['cluster_label'].unique()))}
        )
    )

    print("\nCluster distribution after merging:")
    print(df["cluster_label"].value_counts().sort_index())
    return df


In [ ]:
feature_cols = encoded_df.columns
clustered_df = merge_small_clusters(clustered_df, feature_cols, min_size=20)


In [ ]:
final_clusters = clustered_df[["cluster_label"]].copy()
final_clusters.to_csv("final_cluster_assignments.csv")


In [ ]:
cols = [c for c in clustered_df.columns if c.startswith("cluster_")]
print("Available cluster columns:", cols)
#making sure everything is mapped well
mapping_df = clustered_df.groupby([cols[0], "cluster_label"]).size().reset_index(name="count")
mapping_df = mapping_df.sort_values([cols[0], "count"], ascending=[True, False])
print("\nParent --> child mapping: ")
print(mapping_df)


In [ ]:
parent_child_map = (
    mapping_df[mapping_df["count"] > 0]
    .groupby("cluster_1")["cluster_label"]
    .apply(list)
    .reset_index()
)
parent_child_map.columns = ["Parent_cluster", "Child_clusters"]
print(parent_child_map)


In [ ]:
# Identify which cluster columns exist
cols = [c for c in clustered_df.columns if c.startswith("cluster_")]
print("Cluster columns:", cols)

for i in range(len(cols) - 1):
    print(f"\nLevel {i+1} → Level {i+2} mapping:")
    m = (
        clustered_df.groupby([cols[i], cols[i+1]])
        .size()
        .reset_index(name="count")
        .query("count > 0")
        .sort_values([cols[i], "count"], ascending=[True, False])
    )
    print(m)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

#Supervised UMAP to visualize the clusters. 
le = LabelEncoder()
y = le.fit_transform(clustered_df["cluster_label"])

reducer = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    metric="cosine",
    random_state=42,
    target_metric="categorical",
    target_weight=0.5  
)
embedding = reducer.fit_transform(encoded_df, y=y)
umap_df = pd.DataFrame(embedding, columns=["UMAP1", "UMAP2"])
umap_df["Cluster"] = clustered_df["cluster_label"].values

plt.figure(figsize=(7,6))
sns.scatterplot(
    data=umap_df,
    x="UMAP1", y="UMAP2",
    hue="Cluster",
    s=28, alpha=0.9, linewidth=0, palette="tab10"
)
plt.title("Supervised UMAP (guided by cluster labels)", fontsize=13)
plt.xlabel("UMAP1"); plt.ylabel("UMAP2")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


In [ ]:
# trim the limits, this umap has so much space.
x_low, x_high = np.percentile(umap_df["UMAP1"], [1, 99])
y_low, y_high = np.percentile(umap_df["UMAP2"], [1, 99])
fig, ax = plt.subplots(figsize=(9,8), dpi=300)
sns.scatterplot(
    data=umap_df,
    x="UMAP1", y="UMAP2",
    hue="Cluster",
    palette=palette,
    s=8, alpha=0.85, linewidth=0,
    ax=ax
)
ax.set_xlim(x_low, x_high)
ax.set_ylim(y_low, y_high)
ax.set_aspect('equal', adjustable='box')
ax.set_title("UMAP projection of latent features (C1–C9)", fontsize=14)
ax.set_xlabel("UMAP1", fontsize=12)
ax.set_ylabel("UMAP2", fontsize=12)
ax.legend(
    title="Cluster",
    bbox_to_anchor=(1.12, 1),
    loc="upper left",
    borderaxespad=0.,
    markerscale=1.5
)
fig.tight_layout(pad=0)
plt.subplots_adjust(right=0.85)
fig.savefig("final_figures/UMAP/UMAP_clusters_zoom.png", dpi=600, bbox_inches="tight")
fig.savefig("final_figures/UMAP/UMAP_clusters_zoom.pdf", dpi=600, bbox_inches="tight")
plt.show()


## Analysis

### Hallmark Enrichment

In [ ]:
import gseapy as gp 
import pandas as pd
import numpy as np
from scipy import stats
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
gp.get_library_name(organism='Human')[:200]


In [ ]:
hallmark_sets = gp.get_library(name="MSigDB_Hallmark_2020", organism="Human")
print(f"Loaded {len(hallmark_sets)} Hallmark pathways")


In [ ]:
expr = np.log2(expression_T + 1)
meta = clustered_df.copy()
os.makedirs("DE_lists", exist_ok=True)

# align samples between expression and clusters
common_samples = expr.index.intersection(meta.index)
expr = expr.loc[common_samples].copy()
meta = meta.loc[common_samples].copy()
print(f"{len(common_samples)} samples aligned between expr and clusters.")

# differential expression for each cluster vs all others
de_results = {}

for cl in sorted(meta["cluster_label"].unique()):
    in_group = meta["cluster_label"] == cl
    out_group = ~in_group
    means_in = expr.loc[in_group].mean(axis=0)
    means_out = expr.loc[out_group].mean(axis=0)
    logFC = np.log2((means_in + 1e-6) / (means_out + 1e-6))
    t_stat, p_val = stats.ttest_ind(
        expr.loc[in_group],
        expr.loc[out_group],
        axis=0,
        equal_var=False,
        nan_policy="omit"
    )
    de_df = pd.DataFrame({
        "gene": expr.columns,
        "logFC": logFC,
        "t": t_stat,
        "pval": p_val
    }).sort_values("logFC", ascending=False)

    de_results[cl] = de_df

    de_df[["gene", "logFC"]].to_csv(
        f"DE_lists/{cl}_ranked_genes.rnk",
        sep="\t", index=False, header=False
    )

    print(f"{cl}: saved {len(de_df)} genes to DE_lists/{cl}_ranked_genes.rnk")


In [ ]:
rnk_dir = "DE_lists"
rnk_files = [f for f in os.listdir(rnk_dir) if f.endswith(".rnk")]
gene_sets = "MSigDB_Hallmark_2020"
all_results = []

for file in sorted(rnk_files):
    cluster_id = file.split("_")[0]   
    print(f"Running GSEA for {cluster_id} ...")

    pre_res = gp.prerank(
        rnk=os.path.join(rnk_dir, file),
        gene_sets=gene_sets,
        processes=4,            # number of CPU threads
        permutation_num=1000,    
        outdir=None,           
        seed=42,
        min_size=15,
        max_size=500,
        verbose=False
    )

    df = pre_res.res2d.copy()
    df["Cluster"] = cluster_id
    all_results.append(df)

gsea_results = pd.concat(all_results)
print(f"\nCombined GSEA results shape: {gsea_results.shape}")


In [ ]:
# 1) Inspect what came back from gseapy
print("GSEA results columns:", gsea_results.columns.tolist())
print(gsea_results.head(2))

# 2) Standardize names
df = gsea_results.copy().reset_index()
df.columns = [str(c).strip().lower() for c in df.columns]

# 3) Fix columns and ensure column names are correct
if 'term' not in df.columns:
    if 'name' in df.columns:
        df = df.rename(columns={'name': 'term'})
    elif 'geneset' in df.columns:
        df = df.rename(columns={'geneset': 'term'})
    elif "index" in df.columns:
        df = df.rename(columns={'index': 'term'})
    else:
        raise KeyError(f"Could not find pathway column; available: {df.columns.tolist()}")

#Fix any incorrect or strange naming conventions
if 'nes' not in df.columns:
    alt = [c for c in df.columns if 'nes' in c or 'normalized enrichment' in c]
    if not alt:
        raise KeyError(f"Could not find NES column; available: {df.columns.tolist()}")
    df = df.rename(columns={alt[0]: 'nes'})
    
if 'cluster' not in df.columns and 'Cluster' in gsea_results.columns:
    df = df.rename(columns={'Cluster': "cluster"})

# 4) create the heatmap
heatmap_df = df.pivot_table(index='term', columns='cluster', values='nes').fillna(0)

# order clusters C1-C9 and make pathway names nicer
desired_cols = [f"C{i}" for i in range(1, 10)]
present_cols = [c for c in desired_cols if c in heatmap_df.columns]
heatmap_df = heatmap_df.reindex(columns=present_cols)
heatmap_df.index = (heatmap_df.index
                    .str.replace('HALLMARK_', '', regex=False)
                    .str.replace('_', ' ')
                    .str.title())

print('Heatmap matrix shape:', heatmap_df.shape)
heatmap_df.head()


In [ ]:
plt.figure(figsize=(10, 12))
sns.heatmap(
    heatmap_df,
    cmap="RdBu_r",
    center=0,
    linewidths=0.3,
    linecolor='white',
    cbar_kws={'label': 'NES'}
)
plt.title("Hallmark Pathway Enrichment per Cluster (NES)", fontsize=14)
plt.xlabel("Cluster"); plt.ylabel("Hallmark Pathway")
plt.tight_layout()
fig.savefig("final_figures/GSEA_ranks/Figures/Pathway_enrichment_heatmap.png", dpi=600, bbox_inches="tight")
fig.savefig("final_figures/GSEA_ranks/Figures/Pathway_enrichment_heatmap.pdf", dpi=600, bbox_inches="tight")
plt.show()

In [ ]:
print(len(hallmark_sets))
list(hallmark_sets.keys())[:50]


In [ ]:
paths = [
    "DNA Repair",
    "Myc Targets V2",
    "Reactive Oxygen Species Pathway",
    "Unfolded Protein Response",
    "Cholesterol Homeostasis",
]
[x for x in hallmark_sets.keys() if "dna" in x.lower()]
print(hallmark_sets["DNA Repair"][:200])



In [ ]:
paths = [
    "DNA Repair",
    "Myc Targets V2",
    "Reactive Oxygen Species Pathway",
    "Unfolded Protein Response",
    "Cholesterol Homeostasis",
]

# genes from pathways
genes = sorted({
    g for p in paths
    for g in hallmark_sets.get(p, [])
    if g in expr.columns
})
print(f"{len(genes)} genes found across selected pathways.")

#z-score the exp vals
X = expr[genes].copy()
X_z = (X - X.mean()) / X.std(ddof=0)

#add cluster info and sort by cluster
X_z["cluster"] = clustered_df["cluster_label"]
X_z = X_z.sort_values("cluster")
row_colors = X_z["cluster"].map(palette)

#plot one heatmap per pathway
for p in paths:
    print(f"Plotting: {p}")

    g = [gg for gg in hallmark_sets.get(p, []) if gg in X_z.columns]
    if not g:
        print(f"Skipping {p}: no overlapping genes.")
        continue
    data = X_z[g].T.copy()
    data = data.replace([np.inf, -np.inf], np.nan).dropna(how="any")
    if data.shape[0] < 2:
        print(f" Skipping {p}: not enough variable genes after cleaning.")
        continue
    cg = sns.clustermap(
        data,
        cmap="vlag",
        center=0,
        row_cluster=True,
        col_cluster=False,
        col_colors=row_colors,
        xticklabels=False,
        figsize=(10, 8),
        cbar_kws={'label': 'Z-score'}
    )
    cg.ax_heatmap.set_title(p, fontsize=14, pad=10)
    plt.show()


### Differential Gene Expression

In [ ]:
expr = expr.copy()
clusters = clustered_df["cluster_label"]
unique_clusters = sorted(clusters.unique())
N_TOP = 15  #top/cluster

# one cluster differential expression vs. rest of clusters
de_results = {}
for cl in unique_clusters:
    in_group = clusters[clusters == cl].index
    out_group = clusters[clusters != cl].index

    t_stat, p_val = stats.ttest_ind(expr.loc[in_group], expr.loc[out_group],
                                    equal_var=False, nan_policy="omit")
    logFC = expr.loc[in_group].mean() - expr.loc[out_group].mean()

    df = pd.DataFrame({
        "gene": expr.columns,
        "logFC": logFC,
        "t": t_stat,
        "pval": p_val
    }).sort_values("logFC", ascending=False)

    de_results[cl] = df

top_gene_list = []
cluster_labels_for_rows = []
for cl in unique_clusters:
    genes = de_results[cl].head(N_TOP)["gene"].tolist()
    top_gene_list.extend(genes)
    cluster_labels_for_rows.extend([cl] * len(genes))
top_genes = []
row_clusters = []
for g, cl in zip(top_gene_list, cluster_labels_for_rows):
    if g not in top_genes:
        top_genes.append(g)
        row_clusters.append(cl)
cluster_means = expr.groupby(clusters).mean()[top_genes].T

#z-score per gene 
mean_vals = cluster_means.mean(axis=1).to_numpy()[:, np.newaxis]
std_vals = cluster_means.std(axis=1).to_numpy()[:, np.newaxis]
cluster_means_z = (cluster_means - mean_vals) / std_vals
cluster_means_z.index = pd.MultiIndex.from_arrays(
    [cluster_means_z.index, row_clusters],
    names=["Gene", "TopCluster"]
)

# heatmap with gene labels & cluster boundaries
sns.set(font_scale=0.6)
plt.figure(figsize=(9, max(6, len(top_genes) / 4)))
ax = sns.heatmap(
    cluster_means_z,
    cmap="vlag",
    center=0,
    xticklabels=True,
    yticklabels=[g for g in cluster_means_z.index.get_level_values("Gene")],
    cbar_kws={"label": "Z-score"}
)

# horizontal lines to separate clusters
current = 0
for cl in unique_clusters:
    count = sum(np.array(row_clusters) == cl)
    current += count
    ax.hlines(current, *ax.get_xlim(), color="black", lw=0.6)

ax.set_title("Top Up-regulated Genes per Cluster", fontsize=11, pad=10)
ax.set_xlabel("Cluster")
ax.set_ylabel("Genes (Top 15 per Cluster)")
plt.tight_layout()
plt.savefig("final_figures/DE/fig3A_topgenes_labeled.png", dpi=600)
plt.savefig("final_figures/DE/fig3A_topgenes_labeled.pdf")
plt.show()


### Mutational Signatures

In [ ]:
import shutil
mapping = pd.read_csv("gdc_mapping.tsv", sep="\t")

raw_folder = "extracted/somatic_mutations"
renamed_folder = "stripped/maf"
os.makedirs(renamed_folder, exist_ok=True)

for _, row in mapping.iterrows():
    file_id = row["File_ID"]
    file_name = row["File_Name"]
    case_id = row["Case_ID"]

    # Standardize Case_ID format using first 12 characters (as done previously). 
    case_id = case_id[:12]  

    # copy .maf file
    maf_path = None
    for name in [file_id + ".maf", file_name]:
        path = os.path.join(raw_folder, name)
        if os.path.exists(path):
            maf_path = path
            break

    if maf_path:
        new_name = f"{case_id}.maf"
        shutil.copy(maf_path, os.path.join(renamed_folder, new_name))
    else:
        print(f"File not found for {case_id}")

In [ ]:
from SigProfilerMatrixGenerator import SigProfilerMatrixGenerator as matGen
from SigProfilerExtractor import sigpro as sig

matGen.SigProfilerMatrixGeneratorFunc(
    project="LU_fast",
    reference_genome="GRCh38",
    input_type="stripped/maf",
    exome=True
)

sig.sigProfilerExtractor(
    input_type="matrix",
    input_data="LU_fast.SBS96.exome",
    output="extraction_output",
    reference_genome="GRCh38",
    minimum_signatures=2,
    maximum_signatures=6,
    nmf_replicates=30,
    cpu=2,
    resample=True,
    make_decomposition_plots=True
)


In [ ]:
# Mean COSMIC SBS signature exposures per cluster
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

expo_path = "extraction_output/SBS96/All_Solutions/SBS96_6_Signatures/Activities/SBS96_S6_NMF_Activities.txt"
cluster_path = "final_cluster_assignments.csv"
exp = pd.read_csv(expo_path, sep="\t")
clusters = pd.read_csv(cluster_path)

#fix namings 
clusters.columns = [c.lower() for c in clusters.columns]
sample_col = next((c for c in clusters.columns if "sample" in c), clusters.columns[0])
cluster_col = next((c for c in clusters.columns if "cluster" in c or "label" in c), clusters.columns[1])
clusters = clusters.rename(columns={sample_col:"sample_id", cluster_col:"cluster"})
exp = exp.rename(columns={exp.columns[0]:"sample_id"})

# merge & compute cluster means
merged = exp.merge(clusters, on="sample_id", how="left")
sig_cols = [c for c in exp.columns if c != "sample_id"]
cluster_mean = merged.groupby("cluster")[sig_cols].mean().fillna(0)
ordered = [f"C{i}" for i in range(1,10)]
cluster_mean = cluster_mean.reindex([c for c in ordered if c in cluster_mean.index])

colors = plt.cm.tab10(np.linspace(0,1,len(sig_cols)))
fig, ax = plt.subplots(figsize=(1.5*len(cluster_mean),5))
x = np.arange(len(cluster_mean))
bar_width = 0.12
for i, sig in enumerate(sig_cols):
    ax.bar(x + i*bar_width - bar_width*len(sig_cols)/2,
           cluster_mean[sig], width=bar_width, label=sig, color=colors[i])

ax.set_xticks(x)
ax.set_xticklabels(cluster_mean.index)
ax.set_ylabel("Mean signature exposure")
ax.set_title("Mean COSMIC SBS exposures per cluster", pad=10)
ax.legend(ncol=3, fontsize=8, frameon=False)
plt.tight_layout()
plt.savefig("figures/Fig6F_SignatureExposure_perCluster.png", dpi=300, bbox_inches="tight")
plt.show()

print("Saved: figures/Fig6F_SignatureExposure_perCluster.png")


### Mutation Analysis

In [ ]:
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches

MERGED_PATH = "merged_expression_mutation.csv"
CLUSTERS_CSV = "final_cluster_assignments.csv"
EXOME_MB = 38.0
TOP_N_MUT_GENES = 20

immune_gene_map = {
    "PD1": "PDCD1",
    "PDL1": "CD274",
    "PD-L1": "CD274",
    "PDL2": "PDCD1LG2",
    "PD-L2": "PDCD1LG2"
}
immune_targets = list(dict.fromkeys(immune_gene_map.values()))
base_colors = ["#1f77b4","#ff7f0e","#2ca02c","#d62728","#9467bd",
               "#8c564b","#e377c2","#7f7f7f","#17becf"]
os.makedirs("figures", exist_ok=True)
os.makedirs("results", exist_ok=True)

X = pd.read_csv(MERGED_PATH)
if X.columns[0].lower() in {"sample","sample_id","tumor_sample_barcode","barcode","case_id","unnamed: 0"}:
    X = X.rename(columns={X.columns[0]: "sample_id"}).set_index("sample_id")
else:
    X.index.name = "sample_id"

n_cols = X.shape[1]
half = n_cols // 2
expr_df = X.iloc[:, :half].copy()
mut_df  = X.iloc[:, half:].copy()
expr_genes = expr_df.columns.tolist()
mut_genes  = mut_df.columns.tolist()

clusters = pd.read_csv(CLUSTERS_CSV)
clusters.columns = [c.lower() for c in clusters.columns]
sample_col = next((c for c in ["sample_id","sample","tumor_sample_barcode","barcode","case_id"] if c in clusters.columns), clusters.columns[0])
cluster_col = next((c for c in ["cluster_label","cluster","label","group","subtype"] if c in clusters.columns), None)
if cluster_col is None:
    raise ValueError("No cluster column found.")
clusters = clusters.rename(columns={sample_col:"sample_id", cluster_col:"cluster"}).set_index("sample_id")

expr_df = expr_df.loc[expr_df.index.intersection(clusters.index)]
mut_df  = mut_df.loc[mut_df.index.intersection(clusters.index)]
clusters = clusters.loc[expr_df.index]
ordered_clusters = sorted(clusters["cluster"].unique().tolist())
cluster_to_color = {cl: base_colors[i % len(base_colors)] for i, cl in enumerate(ordered_clusters)}

# TMB 
mut_counts = mut_df.sum(axis=1)
tmb = (mut_counts/EXOME_MB).rename("TMB_perMb").to_frame().join(clusters)
fig, ax = plt.subplots(figsize=(max(6,1.2*len(ordered_clusters)), 4))
data = [tmb[tmb["cluster"]==cl]["TMB_perMb"].values for cl in ordered_clusters]
bp = ax.boxplot(data, labels=ordered_clusters, patch_artist=True)
for i, patch in enumerate(bp["boxes"]):
    patch.set_facecolor(base_colors[i % len(base_colors)])
for i, cl in enumerate(ordered_clusters, start=1):
    y = data[i-1]
    ax.plot(np.random.normal(i, 0.06, size=len(y)), y, "o", alpha=0.4, markersize=3, color="black")
ax.set_ylabel("TMB (mutations/Mb)")
ax.set_title("Tumor Mutational Burden per Cluster")
plt.tight_layout()
plt.savefig("figures/Fig_TMB_PerCluster.png", dpi=300, bbox_inches="tight")
plt.close()
tmb.to_csv("results/tmb_by_sample.tsv", sep="\t")

# oncoplot (top mutated genes)
gene_freq = mut_df.sum(axis=0).sort_values(ascending=False).head(TOP_N_MUT_GENES)
top_genes = gene_freq.index.tolist()
M = mut_df[top_genes].loc[clusters.sort_values("cluster").index]
fig, ax = plt.subplots(figsize=(max(6, 0.4*len(top_genes)), max(6, 0.06*len(M))))
im = ax.imshow(M.values, aspect="auto", cmap=ListedColormap(["#ffffff","#000000"]))
ax.set_xticks(np.arange(len(top_genes)))
ax.set_xticklabels(top_genes, rotation=90, fontsize=8)
ax.set_yticks([])
ax.set_title("Oncoplot-style Heatmap (Top Mutated Genes)", pad=10)
for i, s in enumerate(M.index):
    ax.add_patch(Rectangle((-0.6,i-0.5),0.3,1.0,
                  color=cluster_to_color.get(clusters.loc[s,"cluster"],"#bbbbbb"), lw=0))
patches = [mpatches.Patch(color=cluster_to_color[cl], label=cl) for cl in ordered_clusters]
ax.legend(handles=patches, title="Cluster", bbox_to_anchor=(1.02,1), loc="upper left", frameon=False)
plt.tight_layout()
plt.savefig("figures/Fig_Oncoplot_TopMutatedGenes.png", dpi=300, bbox_inches="tight")
plt.close()

# Stacked Bar Chart
prev = []
for cl in ordered_clusters:
    idx = clusters.index[clusters["cluster"]==cl]
    prev.append((mut_df.loc[idx, top_genes].sum()/len(idx)).rename(cl))
prev_df = pd.concat(prev, axis=1).T
fig, ax = plt.subplots(figsize=(max(8,0.5*len(prev_df.columns)),5))
bottom = np.zeros(len(prev_df))
for g in prev_df.columns:
    vals = prev_df[g].values
    ax.bar(np.arange(len(prev_df)), vals, bottom=bottom, label=g)
    bottom += vals
ax.set_xticks(np.arange(len(prev_df)))
ax.set_xticklabels(prev_df.index)
ax.set_ylabel("Fraction mutated")
ax.set_title("Prevalence of Top Mutated Genes per Cluster")
ax.legend(ncol=min(4,len(prev_df.columns)), fontsize=8, frameon=False, bbox_to_anchor=(1.02,1), loc="upper left")
plt.tight_layout()
plt.savefig("figures/Fig_Stacked_Prevalence_TopMutated_byCluster.png", dpi=300, bbox_inches="tight")
plt.close()
prev_df.to_csv("results/prevalence_top_mutated_per_cluster.tsv", sep="\t")

In [ ]:
# Top 10 most frequently mutated genes/cluster
df = pd.read_csv(MERGED_PATH)
if df.columns[0].lower() in {"sample","sample_id","tumor_sample_barcode","barcode","case_id","unnamed: 0"}:
    df = df.rename(columns={df.columns[0]: "sample_id"}).set_index("sample_id")
else:
    df.index.name = "sample_id"

# mutation frequency/cluster
top_per_cluster = {}
for cl in sorted(clusters["cluster"].unique()):
    idx = clusters.index[clusters["cluster"] == cl]
    sub = mut_df.loc[idx]
    freq = sub.sum(axis=0) / len(sub)
    top10 = freq.sort_values(ascending=False).head(10)
    top_per_cluster[cl] = top10

out_rows = []
for cl, s in top_per_cluster.items():
    for gene, val in s.items():
        out_rows.append({"cluster": cl, "gene": gene, "mutation_fraction": val})
out_df = pd.DataFrame(out_rows)

out_df.to_csv("results/top10_mutated_genes_per_cluster.tsv", sep="\t", index=False)
print("Saved: results/top10_mutated_genes_per_cluster.tsv")


### Immune Analysis

In [ ]:
immune_targets = list(dict.fromkeys(immune_gene_map.values()))
expr_cols = list(expr_df.columns)
expr_present = [g for g in immune_targets if g in expr_cols]
if len(expr_present) < len(immune_targets):
    lower_map = {c.lower():c for c in expr_cols}
    for g in immune_targets:
        if g.lower() in lower_map and lower_map[g.lower()] not in expr_present:
            expr_present.append(lower_map[g.lower()])
#Boxplots
for g in expr_present:
    fig, ax = plt.subplots(figsize=(max(6,1.2*len(ordered_clusters)),4))
    data = [expr_df.loc[clusters["cluster"]==cl, g].values for cl in ordered_clusters]
    bp = ax.boxplot(data, labels=ordered_clusters, patch_artist=True)
    for i,patch in enumerate(bp["boxes"]):
        patch.set_facecolor(base_colors[i % len(base_colors)])
    for i,cl in enumerate(ordered_clusters, start=1):
        y = data[i-1]
        ax.plot(np.random.normal(i,0.06,len(y)), y, "o", alpha=0.35, markersize=3, color="black")
    ax.set_title(f"{g} expression per cluster (log₂(TPM+1))")
    ax.set_ylabel("Expression")
    plt.tight_layout()
    plt.savefig(f"figures/Fig_Box_{g}_perCluster.png", dpi=300, bbox_inches="tight")
    plt.close()

### Clinical Correlations

In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", font_scale=1.0)
plt.rcParams["figure.dpi"] = 120


In [ ]:
clusters = pd.read_csv("final_cluster_assignments.csv", header=None)
clusters.columns = ["Case_ID", "cluster_label"]
clusters["Case_ID"] = clusters["Case_ID"].str[:12].str.upper()

#Clinical Data
clinical = pd.read_csv("clinical.tsv", sep="\t", low_memory=False)
clinical["cases.submitter_id"] = clinical["cases.submitter_id"].astype(str).str[:12].str.upper()
merged = clinical.merge(clusters, how="inner",
                        left_on="cases.submitter_id", right_on="Case_ID")
print("Merge complete:", merged.shape)

#cleanup steps 
clinical_unique = clinical.drop_duplicates(subset=["cases.submitter_id"], keep="first")
merged = clinical_unique.merge(clusters, how="inner",
                               left_on="cases.submitter_id", right_on="Case_ID")
merged.to_csv("merged_clinical_clusters_dedup.csv", index=False)

# ER-ACGR-TTP was missing, no clinical data on it. 
if "ER-ACGR-TTP" not in merged["Case_ID"].values:
    cl = clusters.loc[clusters["Case_ID"].str[:12] == "ER-ACGR-TTP", "cluster_label"].iloc[0]
    merged = pd.concat([merged, pd.DataFrame([{
        "Case_ID": "ER-ACGR-TTP", "cluster_label": cl,
        "cases.submitter_id": "ER-ACGR-TTP",
        "demographic.gender": "Unknown",
        "demographic.race": "Unknown",
        "demographic.age_at_index": None,
        "diagnoses.ajcc_pathologic_stage": "Unknown",
        "cases.disease_type": "Unknown"
    }])], ignore_index=True)

merged.to_csv("merged_clinical_clusters_final.csv", index=False)
print("Saved merged_clinical_clusters_final.csv (n=", len(merged), ")")


In [ ]:
merged = pd.read_csv("merged_clinical_clusters_final.csv")
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

# gender
g = merged["demographic.gender"].fillna("Unknown").value_counts().reset_index()
g.columns = ["Gender","Count"]
sns.barplot(data=g, x="Gender", y="Count",
            palette=["#4C72B0","#F781BF","#BDBDBD"], ax=axes[0])
axes[0].set_title("Gender")

# race
r = merged["demographic.race"].fillna("Unknown").value_counts().reset_index()
r.columns = ["Race","Count"]
sns.barplot(data=r, x="Race", y="Count", palette="Blues", ax=axes[1])
axes[1].set_title("Race / Ethnicity"); axes[1].tick_params(axis="x", rotation=30)

# age
merged["demographic.age_at_index"] = pd.to_numeric(
    merged["demographic.age_at_index"], errors="coerce")
sns.histplot(data=merged, x="demographic.age_at_index", bins=15,
             color="#4C72B0", kde=True, ax=axes[2])
axes[2].set_title("Age at Diagnosis")

# histology
h = merged["cases.disease_type"].fillna("Unknown").value_counts().reset_index()
h.columns = ["Histology","Count"]
sns.barplot(data=h, x="Histology", y="Count",
            palette=["#9ECAE1","#FCAE91","#BDBDBD"], ax=axes[3])
axes[3].set_title("Histopathologic Subtype"); axes[3].tick_params(axis="x", rotation=25)

# stage
s = merged["diagnoses.ajcc_pathologic_stage"].fillna("Unknown").value_counts().reset_index()
s.columns = ["Stage","Count"]
sns.barplot(data=s, x="Stage", y="Count", palette="Greens", ax=axes[4])
axes[4].set_title("AJCC Pathologic Stage"); axes[4].tick_params(axis="x", rotation=30)

# blank for smoking/survival. IDK which one should go there, will manually paste. 
axes[5].axis("off")
axes[5].text(0.5,0.5,"Panel reserved for\nsmoking or survival summary",
             ha="center", va="center", color="gray")

plt.suptitle(f"Figure 1 – Cohort Demographics (n={len(merged)})", fontsize=14, y=1.02)
plt.tight_layout(); plt.savefig("Fig1_Cohort_Demographics.png", dpi=600, bbox_inches="tight"); plt.show()


In [ ]:
#Kaplan-Meier
!pip install lifelines --quiet
from lifelines import KaplanMeierFitter
from lifelines.statistics import multivariate_logrank_test

merged = pd.read_csv("merged_clinical_clusters_final.csv")

merged["status"] = merged["demographic.vital_status"].map({"Alive":0, "Dead":1})
merged["time"] = merged["demographic.days_to_death"].fillna(
    merged["diagnoses.days_to_last_follow_up"])
merged["time"] = pd.to_numeric(merged["time"], errors="coerce")

kmf = KaplanMeierFitter()
palette = sns.color_palette("husl", n_colors=merged["cluster_label"].nunique())

plt.figure(figsize=(7,5))
for i, (cl, subdf) in enumerate(merged.groupby("cluster_label")):
    subdf = subdf.dropna(subset=["time","status"])
    kmf.fit(subdf["time"], event_observed=subdf["status"], label=cl)
    kmf.plot_survival_function(ci_show=False, color=palette[i])
plt.xlabel("Days"); plt.ylabel("Survival probability")
plt.title("Kaplan–Meier Survival by Cluster")
plt.tight_layout(); plt.savefig("Fig_Survival_KM.png", dpi=600); plt.show()

res = multivariate_logrank_test(
    merged["time"], merged["cluster_label"], event_observed=merged["status"])
print("Global log-rank p-value:", res.p_value)


In [ ]:
#Smoking
exposure = pd.read_csv("exposure.tsv", sep="\t")
exposure["cases.submitter_id"] = exposure["cases.submitter_id"].astype(str).str[:12].str.upper()

tobacco = exposure[
    exposure["exposures.tobacco_smoking_status"].notna() |
    exposure["exposures.smoking_frequency"].notna() |
    exposure["exposures.pack_years_smoked"].notna() |
    exposure["exposures.type_of_tobacco_used"].notna()
].copy()

merged = pd.read_csv("merged_clinical_clusters_final.csv")
merged["Case_ID"] = merged["Case_ID"].astype(str).str[:12].str.upper()
merged = merged.merge(
    tobacco[["cases.submitter_id","exposures.tobacco_smoking_status",
             "exposures.pack_years_smoked","exposures.smoking_frequency",
             "exposures.years_smoked","exposures.type_of_tobacco_used"]],
    left_on="Case_ID", right_on="cases.submitter_id", how="left"
)

def simplify_smoking(x):
    if pd.isna(x): return "Unknown"
    x = x.lower()
    if "lifelong" in x or "non" in x or "never" in x: return "Never"
    elif "current" in x and "reformed" not in x: return "Current"
    elif "reformed" in x or "former" in x: return "Former"
    else: return "Other"

merged["smoking_status_clean"] = merged["exposures.tobacco_smoking_status"].apply(simplify_smoking)

summary = (
    merged.groupby(["cluster_label","smoking_status_clean"]).size().reset_index(name="count")
)
summary["percent"] = summary.groupby("cluster_label")["count"].transform(lambda x: x/x.sum()*100)

pivot = summary.pivot(index="cluster_label", columns="smoking_status_clean", values="percent").fillna(0)
order = [c for c in ["Never","Former","Current","Unknown"] if c in pivot.columns]
pivot = pivot[order]

pivot.plot(kind="bar", stacked=True, figsize=(8,4),
           color=["#91bfdb","#fc8d59","#d73027","#d9d9d9"])
plt.ylabel("% of cases"); plt.xlabel("Cluster")
plt.title("Smoking status per cluster")
plt.legend(title="Smoking status", bbox_to_anchor=(1.02,1), loc="upper left")
plt.tight_layout(); plt.savefig("Fig_Smoking_Clusters.png", dpi=600); plt.show()


In [ ]:
#Fixed TMB Figure
M = pd.read_csv("merged_exp_mut.csv", index_col=0)
N_GENES = 19938
mut_cols = M.columns[-N_GENES:]
clusters = pd.read_csv("final_cluster_assignments.csv", header=None, names=["Case_ID","cluster_label"])
clusters["Case_ID"] = clusters["Case_ID"].str[:12].str.upper()
M.index = M.index.str[:12].str.upper()
M = M.merge(clusters, left_index=True, right_on="Case_ID", how="inner")

freq_df = []
for cl in sorted(M["cluster_label"].unique()):
    sub = M[M["cluster_label"]==cl][mut_cols].apply(pd.to_numeric, errors="coerce").fillna(0)
    freq = sub.sum(axis=0)/len(sub)*100
    freq_df.append(freq.rename(cl))
freq_df = pd.DataFrame(freq_df).T

top_genes = freq_df.mean(axis=1).sort_values(ascending=False).head(10).index
plot_df = freq_df.loc[top_genes].T.reset_index().melt(
    id_vars="index", var_name="Gene", value_name="Percent"
).rename(columns={"index":"Cluster"})

fig, ax = plt.subplots(figsize=(8,5))
palette = sns.color_palette("tab10", n_colors=len(top_genes))
bottom = pd.Series([0]*plot_df["Cluster"].nunique(),
                   index=sorted(plot_df["Cluster"].unique()))
for gene, color in zip(top_genes, palette):
    vals = plot_df[plot_df["Gene"]==gene].set_index("Cluster")["Percent"].reindex(bottom.index, fill_value=0)
    ax.bar(bottom.index, vals, bottom=bottom, label=gene, color=color)
    bottom += vals
ax.set_ylabel("Percent of cases mutated (%)")
ax.set_xlabel("Cluster")
ax.set_title("Top mutated genes per cluster")
ax.legend(title="Gene", bbox_to_anchor=(1.05,1), loc="upper left", frameon=False, fontsize=9)
sns.despine(ax=ax)
plt.tight_layout(); plt.savefig("Fig_Mutation_StackedBar.png", dpi=600); plt.show()
